In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("gamma_distr_tofit.csv")

In [3]:
to_fit.head()

,Variable Name,Value,Lower Bound,Upper Bound
0,cLevel1NurseryAdmissionPerDay,1171.750,937.40,1406.10
1,cLevel1NursingCarePerDay,352.000,281.92,422.88
2,cLevel2NICUAdmissionPerDay,5109.250,4087.40,6131.10
3,cLevel2NursingCarePerDay,431.000,344.40,516.60
4,cGentamicin1,34.245,27.40,41.09


In [4]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_gamma_distr(value, lower_bound, upper_bound):

    scale_lower_bound = np.around((abs(lower_bound - value)/2)**2 / value, 2)
    scale_upper_bound = scale_lower_bound + 10
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    scale_lower_bound = int((((upper_bound - value)/1.8)**2 / value) / 100) * 100
    scale_upper_bound = scale_lower_bound + 100000
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    print("No match found")
    return 0, 0

In [5]:
to_fit["a"], to_fit["scale"] = np.vectorize(fit_encompassing_gamma_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

a: 86.09 	 scale: 13.61
a: 86.09 	 scale: 13.61
a: 86.91 	 scale: 4.05
a: 86.12 	 scale: 59.33
a: 85.18 	 scale: 5.06
a: 85.61 	 scale: 0.40
a: 85.93 	 scale: 1.16
a: 86.07 	 scale: 13.21
a: 85.93 	 scale: 3.34
a: 85.71 	 scale: 2.03
a: 86.11 	 scale: 62.46
a: 1.86 	 scale: 223100.00
a: 13.60 	 scale: 84500.00
a: 15.98 	 scale: 637800.00


In [6]:
to_fit["gamma_mean"] = to_fit["a"] * to_fit["scale"]
to_fit["gamma_2.5pct"] = ss.gamma.ppf(0.025, a=to_fit["a"], scale=to_fit["scale"])
to_fit["gamma_97.5pct"] = ss.gamma.ppf(0.975, a=to_fit["a"], scale=to_fit["scale"])

In [7]:
to_fit

,Variable Name,Value,Lower Bound,Upper Bound,a,scale,gamma_mean,gamma_2.5pct,gamma_97.5pct
0,cLevel1NurseryAdmissionPerDay,1.171750e+03,937.40,1406.10,86.094783,13.61,1.171750e+03,9.373705e+02,1.431889e+03
1,cLevel1NursingCarePerDay,3.520000e+02,281.92,422.88,86.913580,4.05,3.520000e+02,2.819047e+02,4.297608e+02
2,cLevel2NICUAdmissionPerDay,5.109250e+03,4087.40,6131.10,86.115793,59.33,5.109250e+03,4.087389e+03,6.243406e+03
3,cLevel2NursingCarePerDay,4.310000e+02,344.40,516.60,85.177866,5.06,4.310000e+02,3.443529e+02,5.272242e+02
4,cGentamicin1,3.424500e+01,27.40,41.09,85.612500,0.40,3.424500e+01,2.737697e+01,4.187011e+01
5,cAmpicillin1,9.968000e+01,79.75,119.62,85.931034,1.16,9.968000e+01,7.972359e+01,1.218320e+02
6,cInitialNICUTests,1.137000e+03,909.60,1364.40,86.071158,13.21,1.137000e+03,9.095420e+02,1.389461e+03
7,cBloodCulture1,2.870000e+02,229.60,344.40,85.928144,3.34,2.870000e+02,2.295403e+02,3.507814e+02
8,cCSF,1.740000e+02,139.20,208.80,85.714286,2.03,1.740000e+02,1.391229e+02,2.127193e+02
9,cReadmission,5.378500e+03,4302.80,6454.19,86.111111,62.46,5.378500e+03,4.302761e+03,6.572458e+03


In [8]:
to_fit.to_csv("gamma_distr_fitted.csv")